<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Cargamos-librerias" data-toc-modified-id="Cargamos-librerias-1">Cargamos librerias</a></span><ul class="toc-item"><li><span><a href="#metricas-de-evaluacion-(sigmas)-+-funciones-de-utilidad" data-toc-modified-id="metricas-de-evaluacion-(sigmas)-+-funciones-de-utilidad-1.1">metricas de evaluacion (sigmas) + funciones de utilidad</a></span></li><li><span><a href="#Datos-de-entrenamiento!" data-toc-modified-id="Datos-de-entrenamiento!-1.2">Datos de entrenamiento!</a></span></li><li><span><a href="#preprocesamiento-para-X-y-Y" data-toc-modified-id="preprocesamiento-para-X-y-Y-1.3">preprocesamiento para X y Y</a></span></li></ul></li><li><span><a href="#Tensorflow" data-toc-modified-id="Tensorflow-2">Tensorflow</a></span></li><li><span><a href="#Keras" data-toc-modified-id="Keras-3">Keras</a></span><ul class="toc-item"><li><span><a href="#Red-sencillita" data-toc-modified-id="Red-sencillita-3.1">Red sencillita</a></span></li><li><span><a href="#Calidad-de-vida:-liveloss!" data-toc-modified-id="Calidad-de-vida:-liveloss!-3.2">Calidad de vida: liveloss!</a></span></li><li><span><a href="#Calidad-de-vida:-agregando-un-tensorboard" data-toc-modified-id="Calidad-de-vida:-agregando-un-tensorboard-3.3">Calidad de vida: agregando un tensorboard</a></span></li><li><span><a href="#Trucos:-Regularizadores-de-redes" data-toc-modified-id="Trucos:-Regularizadores-de-redes-3.4">Trucos: Regularizadores de redes</a></span></li><li><span><a href="#Actividad:-Crea-una-red-profunda" data-toc-modified-id="Actividad:-Crea-una-red-profunda-3.5">Actividad: Crea una red profunda</a></span></li><li><span><a href="#trucos:-Dropout" data-toc-modified-id="trucos:-Dropout-3.6">trucos: Dropout</a></span></li><li><span><a href="#Trucos:-BatchNormalization-y-Activacion!" data-toc-modified-id="Trucos:-BatchNormalization-y-Activacion!-3.7">Trucos: BatchNormalization y Activacion!</a></span></li><li><span><a href="#Keras-Functional" data-toc-modified-id="Keras-Functional-3.8">Keras Functional</a></span></li><li><span><a href="#Truco:-Skip-connections" data-toc-modified-id="Truco:-Skip-connections-3.9">Truco: Skip connections</a></span></li><li><span><a href="#Optimizador" data-toc-modified-id="Optimizador-3.10">Optimizador</a></span></li><li><span><a href="#Controlando-el-learning-rate" data-toc-modified-id="Controlando-el-learning-rate-3.11">Controlando el learning rate</a></span></li><li><span><a href="#Actividad:-Crea-una-red-poderosa!" data-toc-modified-id="Actividad:-Crea-una-red-poderosa!-3.12">Actividad: Crea una red poderosa!</a></span></li></ul></li><li><span><a href="#Pytorch" data-toc-modified-id="Pytorch-4">Pytorch</a></span></li></ul></div>

![](extra/atlas.png)
# Cargamos librerias

In [5]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib
import matplotlib.pyplot as plt

In [6]:
import numpy as np
import scipy as sc
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import os
from IPython.display import display
import sys

## metricas de evaluacion (sigmas) + funciones de utilidad

![http://i.imgur.com/Hflz2lG.jpg](http://i.imgur.com/Hflz2lG.jpg)

In [7]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

    
def AMSScore(s,b): 
    return np.sqrt (2.*( (s + b + 10.)*np.log(1.+s/(b+10.))-s))

def eval_model(Y_true_train,Y_pred_train,w_train,Y_true_test,Y_pred_test,w_test):
    ratio =  float(len(X_train)) /float(len(X_test))
    TruePositive_train = w_train*(Y_true_train==1.0)*(1.0/ratio)
    TrueNegative_train = w_train*(Y_true_train==0.0)*(1.0/ratio)
    TruePositive_valid = w_test*(Y_true_test==1.0)*(1.0/(1-ratio))
    TrueNegative_valid = w_test*(Y_true_test==0.0)*(1.0/(1-ratio))
    s_train = sum ( TruePositive_train*(Y_pred_train==1.0) )
    b_train = sum ( TrueNegative_train*(Y_pred_train==1.0) )
    s_test = sum ( TruePositive_valid*(Y_pred_test==1.0) )
    b_test = sum ( TrueNegative_valid*(Y_pred_test==1.0) )
    score_train = AMSScore(s_train,b_train)
    score_test = AMSScore(s_test,b_test)
    print('--- Resultados --')
    print('- AUC train: {:.3f} '.format(roc_auc_score(Y_train,Y_train_pred)))
    print('- AUC test : {:.3f} '.format(roc_auc_score(Y_test,Y_test_pred)))
    print('- AMS train: {:.3f} sigma'.format(score_train))
    print('- AMS test : {:.3f} sigma'.format(score_test))
    return score_train, score_test

def plot_roc(clf,Y_test,Y_test_prob):
    fpr, tpr, thresholds = roc_curve(Y_test, Y_test_prob)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=1, alpha=0.3, label=str(clf.__class__.__name__))
    plt.plot(np.linspace(0,1,100),np.linspace(0,1,100), lw=2, alpha=0.3, label='Suerte')
    plt.legend(loc='lower right')
    plt.xlim([0,1])
    plt.ylim([0,1])
    plt.tight_layout()
    return

def keras_graph(model):
    from IPython.display import SVG
    from keras.utils.vis_utils import model_to_dot
    return SVG(model_to_dot(model).create(prog='dot', format='svg'))

## Datos de entrenamiento!

Quieres saber mas? Visita

In [8]:
df=pd.read_csv('C:\\Users\\tkalnik\\PycharmProjects\\RIIAA_Escuela18-master\\datos\\training.csv')
print(df.shape)
df.head(1)

(250000, 33)


,EventId,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,...,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt,Weight,Label
0,100000,138.47,51.655,97.827,27.98,0.91,124.711,2.666,3.064,41.928,...,2,67.435,2.15,0.444,46.062,1.24,-2.475,113.497,0.002653,s


## preprocesamiento para X y Y

In [9]:
from sklearn import preprocessing  # to normalize and scale the data
Y = df['Label'].replace(to_replace=['s','b'],value=[1,0]).values
weights = df['Weight'].values
X = df.drop(['EventId','Label','Weight'],axis=1).values

# Did not normalize or scale in final model because AMS results were worse
normalized_X = preprocessing.normalize(X)
scaled_X = preprocessing.scale(X)
scaled_normalized_X = preprocessing.scale(normalized_X)

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test,w_train,w_test = train_test_split(X,Y,weights,train_size=0.3, random_state=42)
print(X_train.shape,Y_train.shape,w_train.shape)
print(X_test.shape,Y_test.shape,w_test.shape)

(75000, 30) (75000,) (75000,)
(175000, 30) (175000,) (175000,)


C:\Users\tkalnik\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


# Tensorflow
[![](extra/tf_logo.jpg)](https://www.tensorflow.org/)

In [11]:
import tensorflow as tf

# Keras

[![](extra/keras-logo.png)](https://keras.io/)

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import backend as K
import keras

## Red sencillita

In [55]:
from keras.layers import Dense, Activation, Dropout

K.clear_session()
tb_call = keras.callbacks.TensorBoard(log_dir='C:\\Users\\tkalnik\\PycharmProjects\\RIIAA_Escuela18-master\\logs3', histogram_freq=0, write_graph=True, write_images=True)

model = Sequential()
model.add(Dense(units=100, input_dim=X_train.shape[-1], kernel_initializer='TruncatedNormal', activation='elu'))
model.add(Dense(units=100, input_dim=X_train.shape[-1], kernel_initializer='TruncatedNormal', activation='elu'))
model.add(Dense(units=100, input_dim=X_train.shape[-1], kernel_initializer='TruncatedNormal', activation='elu'))
model.add(Dense(units=100, input_dim=X_train.shape[-1], kernel_initializer='TruncatedNormal', activation='elu'))
model.add(Dense(units=100, input_dim=X_train.shape[-1], kernel_initializer='TruncatedNormal', activation='elu'))
model.add(Dense(units=100, input_dim=X_train.shape[-1], kernel_initializer='TruncatedNormal', activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(1, kernel_initializer='TruncatedNormal', activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer="Adam", metrics=['accuracy'])
print(model.summary())
model.fit(X_train, Y_train, epochs=170, batch_size=28, callbacks=[tb_call])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               3100      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_6 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_7 (Dense)              (None, 100)               10100     
__________

75000/75000 [==============================] - 7s 89us/step - loss: 0.3802 - acc: 0.8266
Epoch 66/170
75000/75000 [==============================] - 7s 98us/step - loss: 0.3809 - acc: 0.8271
Epoch 67/170
75000/75000 [==============================] - 7s 95us/step - loss: 0.3812 - acc: 0.8276
Epoch 68/170
75000/75000 [==============================] - 8s 108us/step - loss: 0.3819 - acc: 0.8275
Epoch 69/170
75000/75000 [==============================] - 7s 91us/step - loss: 0.3833 - acc: 0.8261
Epoch 70/170
75000/75000 [==============================] - 7s 94us/step - loss: 0.3825 - acc: 0.8285
Epoch 71/170
75000/75000 [==============================] - 8s 109us/step - loss: 0.3832 - acc: 0.8263
Epoch 72/170
 6496/75000 [=>............................] - ETA: 7s - loss: 0.3885 - acc: 0.8196  - ETA: 31s - loss: 0.3

KeyboardInterrupt: 

In [1]:
from sklearn.model_selection import GridSearchCV

In [ ]:
### Grid Search

import numpy as np
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm

def create_model(neurons=1):
    # create model
    model = Sequential()
    model.add(Dense(units=neurons, input_dim=X_train.shape[-1], kernel_initializer='TruncatedNormal', activation='elu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='TruncatedNormal', activation='sigmoid'))
    # compile model
    model.compile(loss='binary_crossentropy', optimizer="Adam", metrics=['accuracy'])
    return(model)

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=1)

# define the grid search parameters for batch size and epochs
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 25, 50, 100, 125, 150]
batch_epoch_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters for the optimizer
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
opt_grid = dict(optimizer=optimizer)

# define the grid search parameters for the initialization weights 
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
init_grid = dict(init_mode=init_mode)

# define the grid search parameters for learning rate and momentum
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
learning_momentum_grid = dict(learn_rate=learn_rate, momentum=momentum)

# define the grid search parameters for neuron activation
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
activation_grid = dict(activation=activation)

# define the grid search parameters
neurons = [1, 5, 10, 15, 20, 25, 30]
neuron_grid = dict(neurons=neurons)

grid = GridSearchCV(estimator=model, param_grid=batch_epoch_grid, n_jobs=-1)
grid_result = grid.fit(X_train, Y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
### Random Search
print(__doc__)

import numpy as np
from time import time
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm

def create_model(neurons=1):
    # create model
    model = Sequential()
    model.add(Dense(units=neurons, input_dim=X_train.shape[-1], kernel_initializer='TruncatedNormal', activation='elu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='TruncatedNormal', activation='sigmoid'))
    # compile model
    model.compile(loss='binary_crossentropy', optimizer="Adam", metrics=['accuracy'])
    return(model)

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=1)

# define the grid search parameters for batch size and epochs
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 25, 50, 100, 125, 150]
batch_epoch_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters for the optimizer
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
opt_grid = dict(optimizer=optimizer)

# define the grid search parameters for the initialization weights 
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
init_grid = dict(init_mode=init_mode)

# define the grid search parameters for learning rate and momentum
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
learning_momentum_grid = dict(learn_rate=learn_rate, momentum=momentum)

# define the grid search parameters for neuron activation
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
activation_grid = dict(activation=activation)

# define the grid search parameters
neurons = [1, 5, 10, 15, 20, 25, 30]
neuron_grid = dict(neurons=neurons)

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(model, param_distributions=batch_epoch_grid,
                                   n_iter=n_iter_search)

start = time()
random_search.fit(X_train, Y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

In [56]:
pcut=0.5
Y_train_pred = model.predict(X_train).flatten() > pcut
Y_test_pred = model.predict(X_test).flatten() > pcut
Y_train_prob= model.predict_proba(X_train).flatten()
Y_test_prob = model.predict_proba(X_test).flatten()
eval_model(Y_train,Y_train_pred,w_train,Y_test,Y_test_pred,w_test)

--- Resultados --
- AUC train: 0.807 
- AUC test : 0.806 
- AMS train: 2.317 sigma
- AMS test : 3.021 sigma


(2.3169095035742773, 3.021130874864326)